In [1]:
# %%
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str, default='InceptionV3')
parser.add_argument('--weights', type=str, default='radimagenet')
parser.add_argument('--n_layers', type=int, default=4)
parser.add_argument('--n_neurons', type=int, default=256)
parser.add_argument('--n_dropout', type=float, default=0.0)
parser.add_argument('--lr_1', type=float, default=3e-4)
parser.add_argument('--lr_2', type=float, default=3e-6)
parser.add_argument('--image_size', type=int, default=512, required=False)
parser.add_argument('--batch_size', type=int, default=16, required=False)

args = parser.parse_args("")

model_name = args.model_name
weights = args.weights
n_layers = args.n_layers
n_neurons = args.n_neurons
n_dropout = args.n_dropout
lr_1 = args.lr_1
lr_2 = args.lr_2
img_size = args.image_size
batch_size = args.batch_size


In [2]:
# %%
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pylab as plt
from sklearn.metrics import roc_curve, roc_auc_score, recall_score
from tensorflow.keras.applications import InceptionResNetV2, ResNet50, InceptionV3, DenseNet121, Xception
import tensorflow_hub as hub
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score
import keras_cv


# %%
train_dir = '../data/split_1/train'
val_dir = '../data/split_1/val'
test_dir = '../data/split_1/test'


# %%
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_dir, label_mode='binary', seed=0, image_size=(img_size, img_size), batch_size=batch_size, color_mode='rgb', crop_to_aspect_ratio=False)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_dir, label_mode='binary', seed=0, image_size=(img_size, img_size), batch_size=batch_size, color_mode='rgb', crop_to_aspect_ratio=False)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_dir, label_mode='binary', seed=0, image_size=(img_size, img_size), batch_size=1, color_mode='rgb', crop_to_aspect_ratio=False)


# %%
#Apply data augmentation
preprocessing_model = tf.keras.Sequential()
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomRotation(40))
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomTranslation(0.2, 0.2))
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2, 0.2))
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal"))
preprocessing_model.add(
    tf.keras.layers.experimental.preprocessing.RandomFlip(mode="vertical"))
#add cutmix augmentation
# preprocessing_model.add(keras_cv.layers.CutMix(1.0))
#add random cutout augmentation
# preprocessing_model.add(keras_cv.layers.RandomCutout(0.5, 0.5))


# %%
train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))


2023-02-16 14:07:03.084010: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 14:07:04.184509: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-16 14:07:04.184595: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-16 14:07:04.184604: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

You do not have pycocotools installed, so KerasCV pycoco metrics are not available. Please run `pip install pycocotools`.
You do not have pyococotools installed, so the `PyCOCOCallback` API is not available.
You do not have Waymo Open Dataset installed, so KerasCV Waymo metrics are not available.
Found 492 files belonging to 2 classes.
Found 165 files belonging to 2 classes.
Found 164 files belonging to 2 classes.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-02-16 14:07:06.905112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 14:07:08.570136: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18266 MB memory:  -> device: 0, name: NVIDIA RTX A4500, pci bus id: 0000:81:00.0, compute capability: 8.6
2023-02-16 14:07:08.570809: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9262 MB memory:  -> device: 1, name: NVIDIA TITAN V, pci bus id: 0000:02:00.0, compute capability: 7.0


In [3]:
if model_name == 'InceptionResNetV2':
    preprocess_fx = tf.keras.applications.inception_resnet_v2.preprocess_input
    model_dir = "../RadImageNet/models/RadImageNet-IRV2_notop.h5"
    if weights == 'imagenet':
        base_model = InceptionResNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
    elif weights == 'radimagenet':
        base_model = InceptionResNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights=model_dir, pooling='avg')
elif model_name == 'ResNet50':
    preprocess_fx = tf.keras.applications.resnet50.preprocess_input
    model_dir = "../RadImageNet/models/RadImageNet-ResNet50_notop.h5"
    if weights == 'imagenet':
        base_model = ResNet50(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
    elif weights == 'radimagenet':
        base_model = ResNet50(input_shape=(img_size, img_size, 3), include_top=False, weights=model_dir, pooling='avg')
elif model_name == 'InceptionV3':
    preprocess_fx = tf.keras.applications.inception_v3.preprocess_input
    model_dir = "../RadImageNet/models/RadImageNet-InceptionV3_notop.h5"
    if weights == 'imagenet':
        base_model = InceptionV3(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
    elif weights == 'radimagenet':
        base_model = InceptionV3(input_shape=(img_size, img_size, 3), include_top=False, weights=model_dir, pooling='avg')
elif model_name == 'DenseNet121':
    preprocess_fx = tf.keras.applications.densenet.preprocess_input
    model_dir = "../RadImageNet/models/RadImageNet-DenseNet121_notop.h5"
    if weights == 'imagenet':
        base_model = DenseNet121(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
    elif weights == 'radimagenet':
        base_model = DenseNet121(input_shape=(img_size, img_size, 3), include_top=False, weights=model_dir, pooling='avg')
elif model_name == 'Xception':
    preprocess_fx = tf.keras.applications.xception.preprocess_input
    if weights == 'imagenet':
        base_model = tf.keras.applications.Xception(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet', pooling='avg')
elif model_name == 'BiT':
    base_model = hub.KerasLayer("https://tfhub.dev/google/bit/m-r50x1/1", trainable=False)
    preprocess_fx = tf.keras.applications.resnet50.preprocess_input
    


In [4]:


inputs = keras.Input(shape=(img_size, img_size, 3))
y = preprocess_fx(inputs)
y = base_model(y, training=False)
for i in range(n_layers):
    y = keras.layers.Dense(n_neurons, activation='relu')(y)
    y = keras.layers.Dropout(n_dropout)(y)
outputs = keras.layers.Dense(1, activation='sigmoid')(y)
model = keras.Model(inputs, outputs)

# %%
early_stopping = keras.callbacks.EarlyStopping(patience=50, min_delta=1e-10, restore_best_weights=True)

# %%

model.compile(
    optimizer=keras.optimizers.Adam(lr_1), 
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
)

epochs = 1000


In [5]:
model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=early_stopping, verbose=1)
print('phase 1 complete')
# %%
#unfreeze all layers and train at lower learning rate
base_model.trainable = True
model.compile(
    optimizer=keras.optimizers.Adam(lr_2), 
    loss=keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=[keras.metrics.BinaryAccuracy(), keras.metrics.AUC()],
)
model.fit(train_ds, epochs=1000, validation_data=val_ds, callbacks=early_stopping, verbose=1)
print('phase 2 complete')


Epoch 1/1000


2023-02-16 14:07:38.403212: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8800
2023-02-16 14:07:40.437870: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-02-16 14:07:40.440511: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2023-02-16 14:07:40.440527: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:237] Used ptxas at ptxas
2023-02-16 14:07:40.440621: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation.

 1/31 [..............................] - ETA: 22:11 - loss: 0.6974 - binary_accuracy: 0.3125 - auc: 0.3750

2023-02-16 14:08:03.050426: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


31/31 [==============================] - 71s 896ms/step - loss: 0.6950 - binary_accuracy: 0.5142 - auc: 0.4972 - val_loss: 0.6909 - val_binary_accuracy: 0.5273 - val_auc: 0.5811
Epoch 2/1000
31/31 [==============================] - 23s 731ms/step - loss: 0.6932 - binary_accuracy: 0.5000 - auc: 0.4886 - val_loss: 0.6914 - val_binary_accuracy: 0.5273 - val_auc: 0.5339
Epoch 3/1000
31/31 [==============================] - 23s 723ms/step - loss: 0.6861 - binary_accuracy: 0.5467 - auc: 0.5835 - val_loss: 0.7214 - val_binary_accuracy: 0.5273 - val_auc: 0.5681
Epoch 4/1000
31/31 [==============================] - 23s 729ms/step - loss: 0.6960 - binary_accuracy: 0.5285 - auc: 0.5257 - val_loss: 0.6918 - val_binary_accuracy: 0.5576 - val_auc: 0.5575
Epoch 5/1000
31/31 [==============================] - 23s 723ms/step - loss: 0.6859 - binary_accuracy: 0.5650 - auc: 0.5784 - val_loss: 0.6919 - val_binary_accuracy: 0.5515 - val_auc: 0.5680
Epoch 6/1000
31/31 [==============================] - 24s 

In [6]:
#save the final model as a .h5 file
model.save('models/{}_{}_cutout.h5'.format(model_name, weights))

In [7]:
#generate predictions for all images in the dataset, including train and val, and save to one csv
train_ds = tf.keras.preprocessing.image_dataset_from_directory(train_dir, label_mode='binary', seed=0, shuffle=False, image_size=(img_size, img_size), batch_size=batch_size, color_mode='rgb')
val_ds = tf.keras.preprocessing.image_dataset_from_directory(val_dir, label_mode='binary', seed=0, shuffle=False, image_size=(img_size, img_size), batch_size=batch_size, color_mode='rgb')
test_ds = tf.keras.preprocessing.image_dataset_from_directory(test_dir, label_mode='binary', seed=0, shuffle=False, image_size=(img_size, img_size), batch_size=1, color_mode='rgb')
y_train_pred = np.array([])
y_train = np.array([])
for images, labels in train_ds:
    y_train = np.append(y_train, labels.numpy())
    y_train_pred = np.append(y_train_pred, model.predict(images))
train_file_paths = train_ds.file_paths

y_val_pred = np.array([])
y_val = np.array([])
for images, labels in val_ds:
    y_val = np.append(y_val, labels.numpy())
    y_val_pred = np.append(y_val_pred, model.predict(images))
val_file_paths = val_ds.file_paths

y_test_pred = np.array([])
y_test = np.array([])
for images, labels in test_ds:
    y_test = np.append(y_test, labels.numpy())
    y_test_pred = np.append(y_test_pred, model.predict(images))
test_file_paths = test_ds.file_paths

y_all_pred = np.append(y_train_pred, y_val_pred)
y_all_pred = np.append(y_all_pred, y_test_pred)
y_all = np.append(y_train, y_val)
y_all = np.append(y_all, y_test)
file_paths = np.append(train_file_paths, val_file_paths)
file_paths = np.append(file_paths, test_file_paths)
pred_df = pd.DataFrame({'file_path': file_paths, 'y_true': y_all, 'y_pred': y_all_pred})
pred_df.to_csv('../preds/preds_{}_{}_{}_{}_all.csv'.format(model_name, weights, n_layers, n_neurons), index=False)


Found 492 files belonging to 2 classes.
Found 165 files belonging to 2 classes.
Found 164 files belonging to 2 classes.
1/1 [==============================] - 0s 39ms/step


In [8]:
auroc_test = roc_auc_score(y_test, y_test_pred)
auroc_val = roc_auc_score(y_val, y_val_pred)
print("test auroc: {}".format(auroc_test))
print("val auroc: {}".format(auroc_val))


test auroc: 0.7069171138938581
val auroc: 0.6248157972295904


In [9]:
results_df = pd.DataFrame({'model': [model_name], 'weights': [weights], 'image_size': [img_size], 'batch_size': [batch_size], 'lr_1': [lr_1], 'lr_2': [lr_2], 'n_layers': [n_layers], 'n_neurons': [n_neurons], 'n_dropout': [n_dropout], 'auroc_val': auroc_val, 'auroc_test': auroc_test})